In [10]:
from scipy.spatial import ConvexHull
import numpy as np
import pandas as pd
from geopy import distance
from ortools.linear_solver import pywraplp

# City for new hub
city = 'Surrey'

# Number of new hubs
num_hubs = 5

# Import potential partner info
charity_info = pd.read_csv('charities_latlong.csv')

# Get address parts
charity_info[['street','city','province','postcode','country']] = charity_info['full_address'].str.split(', ', n=4, expand=True)

# Transform city to lowercase for matching
charity_info['city'] = charity_info['city'].str.lower()

# Filter charities by target city
city_charities = charity_info[charity_info.city.eq(city.lower())]

# Get numpy array of coordinates
charity_coord_arr = city_charities[['Lat','Long']].to_numpy()

# Get distances between each charity
charity_coord_distances = np.zeros((len(charity_coord_arr),len(charity_coord_arr)))

#print(charity_coord_arr)
for i in range(len(charity_coord_arr)):
    lat1, long1 = charity_coord_arr[i]
    for j in range(len(charity_coord_arr)):
        lat2, long2 = charity_coord_arr[j]
        charity_coord_distances[i][j] = distance.distance((lat1,long1),(lat2,long2)).m

# Get convex hull of locations in city
city_hull = ConvexHull(charity_coord_arr)

# Number of charities in city
num_charities = len(charity_coord_arr)

# Initialize integer programming solver
solver = solver = pywraplp.Solver('Stewards', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# Initialize variables: value of 1 means this charity will be a steward, 0 means
# It won't be
X = {}

for i in range(num_charities):
    for j in range(num_hubs):
        X[i,j] = solver.BoolVar(f'X[{i},{j}]')

# Add constraints: only 1 charity per steward
for j in range(num_hubs):
    solver.Add(solver.Sum([X[i,j] for i in range(num_charities)]) == 1)

# Add constraints: only 1 hub per charity
for i in range(num_charities):
    solver.Add(solver.Sum([X[i,j] for j in range(num_hubs)]) <= 1)

# Objective function: minimize average distance from outer edge of city
solver.Minimize(solver.Sum([charity_coord_distances[i][d]*X[i,j] for i in range(num_charities) for j in range(num_hubs) for d in city_hull.vertices])/len(city_hull.vertices) + 
    solver.Sum([charity_coord_distances[i][y]*X[i,j] for i in range(num_charities) for y in range(num_charities) for j in range(num_hubs)]))

solver.Solve()

# Print best-located charities
for i in range(num_charities):
    for j in range(num_hubs):
        if X[i,j].solution_value() == 1:
            print(city_charities.iloc[i])

Legal name                    NEW HOPE THERAPEUTIC SOCIETY
full_address    204-13911 70TH AVE, SURREY, BC, V3W6B4, CA
Lat                                              49.130265
Long                                           -122.835875
street                                  204-13911 70TH AVE
city                                                surrey
province                                                BC
postcode                                            V3W6B4
country                                                 CA
Name: 725, dtype: object
Legal name                           BHALAEE FOUNDATION
full_address    14927-72 AVENUE, SURREY, BC, V3S2E9, CA
Lat                                           49.134026
Long                                        -122.808988
street                                  14927-72 AVENUE
city                                             surrey
province                                             BC
postcode                                         V3S